In [8]:
%load_ext autoreload
%autoreload 2

from ProblemEvaluator import ProblemEvaluator
import numpy as np
from pymoo.problems import get_problem
import matplotlib.pyplot as plt
from optimisation.model.population import Population
from features.globalfeatures import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Flags for which feature set we want to test.
sample_rw = True
sample_aw = False

# Example problem.
num_samples = 2
n_var = 10
problem_name = "MW12"
problem = get_problem(problem_name, n_var)
instance_string = f"{problem_name}_d{n_var}"

# Generate evaluator which can generate RW samples and LHS samples.
evaluator = ProblemEvaluator(problem, instance_string, "eval", "test_results", 1)
pre_sampler = evaluator.create_pre_sampler(num_samples)
pre_sampler.create_pregen_sample_dir()
pre_sampler.create_pops_dir(instance_string)

Initialising evaluator in eval mode.
Directories created or refreshed:
Global: ../evaluated_pops\MW12_d10\eval\global
RW: ../evaluated_pops\MW12_d10\eval\rw


In [10]:
import time
import pickle
import numpy as np
import os

# Assuming pre_sampler, evaluator, and problem are already defined
# and generate_global_population method returns pop_global as a numpy array or a list that can be converted to a numpy array

# Generate distributed samples and evaluate populations on these samples.
sample = pre_sampler.read_global_sample(5)
start_time = time.time()
pop_global = evaluator.generate_global_population(
    problem, sample, eval_fronts=False
)
end_time = time.time()

evaluation_time = end_time - start_time  # Calculate total evaluation time in seconds
print(f"Total evaluation time: {evaluation_time:.2f} seconds, Sample size: {len(pop_global)}")

# Save with pickle
pickle_start_time = time.time()
with open("pop_global.pkl", "wb") as file:
    pickle.dump(pop_global, file)
pickle_end_time = time.time()
pickle_time = pickle_end_time - pickle_start_time

# Save with numpy
numpy_start_time = time.time()
np.save("pop_global.npy", pop_global)
numpy_end_time = time.time()
numpy_time = numpy_end_time - numpy_start_time

print(f"Time to save with pickle: {pickle_time:.2f} seconds")
print(f"Time to save with numpy: {numpy_time:.2f} seconds")


Total evaluation time: 47.82 seconds, Sample size: 10000
Time to save with pickle: 0.12 seconds
Time to save with numpy: 0.14 seconds


In [12]:
import time
import pickle
import numpy as np

# Read with pickle
pickle_read_start_time = time.time()
with open("pop_global.pkl", "rb") as file:
    pop_global_pickle = pickle.load(file)
pickle_read_end_time = time.time()
pickle_read_time = pickle_read_end_time - pickle_read_start_time

print(f"Time to read with pickle: {pickle_read_time:.2f} seconds")


Time to read with pickle: 0.13 seconds


In [ ]:
# IC features
start_ic_time = time.time()

# Compute IC features
(H_max, eps_s, m0, eps05) = compute_ic_features(pop_global)

# End time for computing IC features
end_ic_time = time.time()

# Calculate the computation time
ic_computation_time = end_ic_time - start_ic_time

# Get the size of pop_global
sample_size = len(pop_global)

# Print the descriptive message
print(f"Computed IC features in {ic_computation_time:.2f} seconds. Sample size: {sample_size}.")

Computed IC features in 3.93 seconds. Sample size: 10000.
